In [1]:
%load_ext chime
import os
import geopandas as gpd
import rasterio as rio
import gdal
from glob import glob
import numpy as np
from skimage.morphology import remove_small_holes
from utils import finished, getSentinelBandFile
from shapely.geometry import box

In [2]:
sentinel_dir = r"R:\ProjectData\PAG2019\Sentinel2Data"

sindicies = ["NDWI", "BSI", "NDVI", "NDVIa", "DBSI", "NDMI"]
sbands = [2, 3, 4, 5, 6, 7, 8, "8A", 11, 12]

sentinel_bands = {}
bd = {}

for b in sbands:
    band = f"B0{b}" if len(str(b)) == 1 else f"B{b}"
    sentinel_bands[b] = getSentinelBandFile(band, sentinel_dir, suffix="20190901")

warpOpts = gdal.WarpOptions(dstSRS="EPSG:2868")

for band, band_loc in sentinel_bands.copy().items():
    bandRepro_loc = band_loc.replace(".tif","_2868.tif")
    tif = gdal.Warp(bandRepro_loc, band_loc, options=warpOpts)
    del tif
    sentinel_bands[band] = bandRepro_loc
sentinel_bands

{2: 'R:\\ProjectData\\PAG2019\\Sentinel2Data\\B02Mean_20190601to20190901_2868.tif',
 3: 'R:\\ProjectData\\PAG2019\\Sentinel2Data\\B03Mean_20190601to20190901_2868.tif',
 4: 'R:\\ProjectData\\PAG2019\\Sentinel2Data\\B04Mean_20190601to20190901_2868.tif',
 5: 'R:\\ProjectData\\PAG2019\\Sentinel2Data\\B05Mean_20190601to20190901_2868.tif',
 6: 'R:\\ProjectData\\PAG2019\\Sentinel2Data\\B06Mean_20190601to20190901_2868.tif',
 7: 'R:\\ProjectData\\PAG2019\\Sentinel2Data\\B07Mean_20190601to20190901_2868.tif',
 8: 'R:\\ProjectData\\PAG2019\\Sentinel2Data\\B08Mean_20190601to20190901_2868.tif',
 '8A': 'R:\\ProjectData\\PAG2019\\Sentinel2Data\\B8AMean_20190601to20190901_2868.tif',
 11: 'R:\\ProjectData\\PAG2019\\Sentinel2Data\\B11Mean_20190601to20190901_2868.tif',
 12: 'R:\\ProjectData\\PAG2019\\Sentinel2Data\\B12Mean_20190601to20190901_2868.tif'}

In [3]:
## There still exist some zero (no data values scattered around ponds that didn't get filled/filterd in the intial process. Had to go back and fix (one-time))
#bandMeanFiles = [r"G:\OrthophotosAndElevation\2019\Sentinel2Data\B8AMean_20190601to20190901.tif"]
#with rio.open(bandMeanFiles[0]) as src:
#    data, mask = src.read(1, masked=True)
#    print(len(data[np.isnan(data)]))
for band, band_loc in sentinel_bands.items():
    with rio.open(band_loc) as src:
        data = src.read(1)
        kwargs = src.profile

    filled = remove_small_holes(data!=0, area_threshold=100, connectivity=3)
    data[data==0 & filled ] = 1
    data[~filled] = 0

    with rio.open(band_loc, "w", **kwargs) as dst:
        dst.write(data, 1)
    print(f"Filled holes in {band_loc}")

Filled holes in R:\ProjectData\PAG2019\Sentinel2Data\B02Mean_20190601to20190901_2868.tif
Filled holes in R:\ProjectData\PAG2019\Sentinel2Data\B03Mean_20190601to20190901_2868.tif
Filled holes in R:\ProjectData\PAG2019\Sentinel2Data\B04Mean_20190601to20190901_2868.tif
Filled holes in R:\ProjectData\PAG2019\Sentinel2Data\B05Mean_20190601to20190901_2868.tif
Filled holes in R:\ProjectData\PAG2019\Sentinel2Data\B06Mean_20190601to20190901_2868.tif
Filled holes in R:\ProjectData\PAG2019\Sentinel2Data\B07Mean_20190601to20190901_2868.tif
Filled holes in R:\ProjectData\PAG2019\Sentinel2Data\B08Mean_20190601to20190901_2868.tif
Filled holes in R:\ProjectData\PAG2019\Sentinel2Data\B8AMean_20190601to20190901_2868.tif
Filled holes in R:\ProjectData\PAG2019\Sentinel2Data\B11Mean_20190601to20190901_2868.tif
Filled holes in R:\ProjectData\PAG2019\Sentinel2Data\B12Mean_20190601to20190901_2868.tif


In [4]:
bd = {}
for band, band_loc in sentinel_bands.items():
    with rio.open(band_loc) as src:
        data = src.read(1).astype(np.float32)
        data[data==src.nodata] = np.nan
        bd[band] = data/10000.0

with rio.open(sentinel_bands[2]) as src:
    kwargs = src.profile
kwargs.update(dtype=np.float32, nodata=-9999)

# Normalized Difference Vegetation Index
with rio.open(sentinel_bands[2].replace("B02Mean", "NDVI"), "w", **kwargs) as dst:
    ndvi = (bd[8]-bd[4])/(bd[8]+bd[4])
    dst.write(ndvi, 1)

# Normalized Difference Vegetation Index (Band 8A)
with rio.open(sentinel_bands[2].replace("B02Mean", "NDVIa"), "w", **kwargs) as dst:
    ndvia =(bd["8A"]-bd[4])/(bd["8A"]+bd[4])
    dst.write(ndvia, 1)

# Normalized Difference Water Index
with rio.open(sentinel_bands[2].replace("B02Mean", "NDWI"), "w", **kwargs) as dst:
    ndwi = (bd[3]-bd[8])/(bd[3]+bd[8])
    dst.write(ndwi, 1)
    
# Bare Soil Index
with rio.open(sentinel_bands[2].replace("B02Mean", "BSI"), "w", **kwargs) as dst:
    bsi  = ((bd[11]+bd[4])-(bd[8]+bd[2]))/((bd[11]+bd[4])+(bd[8]+bd[2]))
    dst.write(bsi, 1)

# Dry Bare Soil Index
with rio.open(sentinel_bands[2].replace("B02Mean", "DBSI"), "w", **kwargs) as dst:
    dbsi = (bd[11]-bd[3])/(bd[11]+bd[3])
    dst.write(dbsi, 1)

# Normalized Difference Moisture Index
with rio.open(sentinel_bands[2].replace("B02Mean", "NDMI"), "w", **kwargs) as dst:
    ndmi = (bd[8]-bd[11])/(bd[8]+bd[11])
    dst.write(ndmi, 1)


finished()

In [5]:
%chime

In [15]:
#ndvi = (b08-b04)/(b08+b04)
#ndvia =(b8a-b04)/(b8a+b04)
#bsi  = ((b11-b04)-(b08+b02))/((b11+b04)+(b08+b02))
#ndwi = (b03-b08)/(b03+b08)
#dbsi = (b11-b03)/(b11+b03)
#ndmi = (b08-b11)/(b08+b11)